In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from datasets import load_from_disk, load_metric
import argparse
import os

# Configuration
MODEL_NAME = "google/pegasus-cnn_dailymail"
BATCH_SIZE = 1
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_OUTPUT_LENGTH = 150

def parse_args():
    parser = argparse.ArgumentParser(description='Summarization with PEGASUS')
    parser.add_argument('--output_path', type=str, required=True, help='Path to store evaluation results.')
    parser.add_argument('--filename', type=str, required=True, help='Filename to store evaluation results.')
    return parser.parse_args()

# Load the tokenizer and model
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)

# Custom Dataset class for DataLoader
class CNNDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(self.encodings["labels"][idx])
        }
        return item

# Training function
def train_model(dataset):
    # Load and preprocess data
    dataloader = DataLoader(CNNDataset(dataset), batch_size=BATCH_SIZE, shuffle=True)
    
    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    # Training loop
    model.train()
    for epoch in range(3):  # Number of epochs can be adjusted
        for i,batch in enumerate(dataloader):
            # Move batch to device
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Log the loss value
            if (i % (len(dataloader)/BATCH_SIZE//20) == 0):
                print(f'Epoch: {epoch}, {i/len(dataloader)}, Loss: {loss.item()}')

# Evaluation function
def evaluate_model(output_path, dataset):
    metric = load_metric('rouge')
    dataloader = DataLoader(CNNDataset(dataset), batch_size=BATCH_SIZE)

    model.eval()
    for batch in dataloader:
        # Move batch to device
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        
        # Generate summaries
        with torch.no_grad():
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=MAX_OUTPUT_LENGTH)
        
        # Decode generated summaries and references
        generated_summaries = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        references = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

        # Add predictions and references to compute ROUGE scores
        metric.add_batch(predictions=generated_summaries, references=references)

    # Compute and print ROUGE scores
    scores = metric.compute()
    print(f"ROUGE scores: {scores}")

    # Save the scores to a file
    
    with open(os.path.join(output_path, "result.txt"), 'w') as f:
        f.write(str(scores))

# Main execution
if __name__ == "__main__":
    
    args = parse_args()
    output_path = args.output_path
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    to_store = os.path.join(args.output_path, args.filename)
    
    train_dt = load_from_disk("F:/NLP/data/CNN/dataset_train")
    test_dt = load_from_disk("F:/NLP/data/CNN/dataset_test")
    try:
        train_model(dataset=train_dt)
    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"Batch size: {BATCH_SIZE} needs to be decreased.")
        else:
            raise  # Re-raise the exception if it's not related to out of memory
    evaluate_model(output_path=to_store, dataset=test_dt)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, 0.0, Loss: 8.818455696105957


KeyboardInterrupt: 